In [1]:
# 引入相关的库
import json
import codecs
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import codecs
from tqdm import tqdm
import jieba
from gensim.models import Word2Vec, KeyedVectors

import dgl
from dgl.dataloading import GraphDataLoader
from dgl.data import DGLDataset
from dgl.nn.pytorch import GraphConv, GATConv
from dgl.nn import AvgPooling, MaxPooling

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import pickle

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score

# 加载数据

In [2]:
# 设置源数据文件夹路径
src_data_folder = './data'

# 定义读取数据的函数get_data
def get_data(file_path, flag=None): 
    """读取数据"""
    text = []
    target = []
    with codecs.open(file_path, 'r', encoding='utf-8') as fin:  
        for line in fin:                                        
            tmp_line = json.loads(line)
            text.append(tmp_line["fact"].strip())               
            target.append(tmp_line["meta"]["accusation"][0])    
    return pd.DataFrame({'text': text, 'target': target})       

In [3]:
print('读取数据...')
# 使用os.path.join构造完整的文件路径。
# 调用get_data函数读取每个数据集，并将结果存储在变量train_df、valid_df和test_df中。
train_df = get_data(os.path.join(src_data_folder, 'data_train.json'), flag='train')
valid_df = get_data(os.path.join(src_data_folder, 'data_valid.json'), flag='valid')
test_df = get_data(os.path.join(src_data_folder, 'data_test.json'), flag='test')

print('读取完成！')

读取数据...
读取完成！


In [4]:
# 查看训练集、验证集和测试集各自的样本数量，你可以直接运行上述代码片段。这里，shape属性返回一个元组，表示DataFrame的维度（行数，列数）
train_df.shape, valid_df.shape, test_df.shape

((154592, 2), (17131, 2), (32508, 2))

In [5]:
# 训练集前五行
train_df.head()

,text,target
0,昌宁县人民检察院指控，2014年4月19日下午16时许，被告人段某驾拖车经过鸡飞乡澡塘街子，...,故意伤害
1,"公诉机关指控,2015年11月10日晚9时许，被告人李某的妹妹李某某与被害人华某某在桦川县悦...",故意伤害
2,贵州省平坝县人民检察院指控：2014年4月9日下午，被告人王某丁与其堂哥王4某（另案处理）假...,故意伤害
3,经审理查明：2014年5月6日14时许，被告人叶某某驾车途径赤壁市赵李桥镇胜利街涵洞时，被在...,故意伤害
4,安阳县人民检察院指控：2014年4月27日上午11时许，宋某甲在安阳县吕村镇翟奇务村被告人梁...,故意伤害


In [6]:
# 验证集前五行
valid_df.head()

,text,target
0,公诉机关起诉指控，被告人张某某秘密窃取他人财物，价值2210元，××数额较大，其行为已触犯《...,盗窃
1,孝昌县人民检察院指控：2014年1月4日，被告人邬某在孝昌县城区2路公交车上××被害人晏某白...,盗窃
2,广东省广州市南沙区人民检察院指控被告人罗某于2015年6月2日到广州市南沙区大岗镇人民路宇航...,盗窃
3,公诉机关指控，2016年3月3日18时许，被告人易某某行至达州市通川区大观园公交车站附近，扒...,盗窃
4,公诉机关指控：1.2015年8月20日晚上，被告人胡某甲至杭州市淳安县千岛湖镇新安东路112...,盗窃


In [7]:
# 测试集前五行
test_df.head()

,text,target
0,公诉机关指控：2016年3月28日20时许，被告人颜某在本市洪山区马湖新村足球场马路边捡拾到...,盗窃
1,天津市静海县人民检察院指控，2014年5月13日上午8时许，被告人李xx在天津市静海县大邱庄...,盗窃
2,永顺县人民检察院指控，2014年1月11日，被告人李某某与彭某某（另案处理）在永顺县塔卧镇“...,强奸
3,公诉机关起诉书指控：2016年11月17日凌晨1时许，被告人周某在本县武康街道营盘小区131...,盗窃
4,大名县人民检察院起诉书指控，2014年3月25日9时许，被告人张某在自家庄某处因故与本村席某...,故意伤害


In [8]:
# 获取所有唯一标签
labels = train_df['target'].unique()    
print('总标签数: ', len(labels))

# 创建标签到ID的映射
label2idx = {l:i for i,l in enumerate(sorted(labels))}   
idx2label = {v:k for k,v in label2idx.items()}           

# 使用apply()方法和之前创建的label2idx字典，将三个数据集中'target'列的标签替换为其对应的整数ID。
# 将标签转换为ID
train_df['target'] = train_df['target'].apply(lambda x: label2idx[x])
valid_df['target'] = valid_df['target'].apply(lambda x: label2idx[x])
test_df['target'] = test_df['target'].apply(lambda x: label2idx[x])

总标签数:  195


In [9]:
# 检查训练集前5条数据
train_df.head()

,text,target
0,昌宁县人民检察院指控，2014年4月19日下午16时许，被告人段某驾拖车经过鸡飞乡澡塘街子，...,95
1,"公诉机关指控,2015年11月10日晚9时许，被告人李某的妹妹李某某与被害人华某某在桦川县悦...",95
2,贵州省平坝县人民检察院指控：2014年4月9日下午，被告人王某丁与其堂哥王4某（另案处理）假...,95
3,经审理查明：2014年5月6日14时许，被告人叶某某驾车途径赤壁市赵李桥镇胜利街涵洞时，被在...,95
4,安阳县人民检察院指控：2014年4月27日上午11时许，宋某甲在安阳县吕村镇翟奇务村被告人梁...,95


In [10]:
# 检查验证集前5条数据
valid_df.head()

,text,target
0,公诉机关起诉指控，被告人张某某秘密窃取他人财物，价值2210元，××数额较大，其行为已触犯《...,108
1,孝昌县人民检察院指控：2014年1月4日，被告人邬某在孝昌县城区2路公交车上××被害人晏某白...,108
2,广东省广州市南沙区人民检察院指控被告人罗某于2015年6月2日到广州市南沙区大岗镇人民路宇航...,108
3,公诉机关指控，2016年3月3日18时许，被告人易某某行至达州市通川区大观园公交车站附近，扒...,108
4,公诉机关指控：1.2015年8月20日晚上，被告人胡某甲至杭州市淳安县千岛湖镇新安东路112...,108


In [11]:
# 检查测试集前5条数据
test_df.head()

,text,target
0,公诉机关指控：2016年3月28日20时许，被告人颜某在本市洪山区马湖新村足球场马路边捡拾到...,108
1,天津市静海县人民检察院指控，2014年5月13日上午8时许，被告人李xx在天津市静海县大邱庄...,108
2,永顺县人民检察院指控，2014年1月11日，被告人李某某与彭某某（另案处理）在永顺县塔卧镇“...,70
3,公诉机关起诉书指控：2016年11月17日凌晨1时许，被告人周某在本县武康街道营盘小区131...,108
4,大名县人民检察院起诉书指控，2014年3月25日9时许，被告人张某在自家庄某处因故与本村席某...,95


In [12]:
# 罪名与id之间的映射,从罪名到其对应编号的映射关系
a=pd.DataFrame(list(label2idx.items()),columns=['罪名', '编号'])
a

,罪名,编号
0,[伪造、倒卖]伪造的有价票证,0
1,[伪造、变造]居民身份证,1
2,[伪造、变造]金融票证,2
3,[伪造、变造、买卖]国家机关[公文、证件、印章],3
4,[伪造、变造、买卖]武装部队[公文、证件、印章],4
...,...,...
190,非法行医,190
191,非法进行节育手术,191
192,非法采矿,192
193,骗取[贷款、票据承兑、金融票证],193


# 分词和去停用词

In [13]:
# 定义停用词文件的路径STOPWORDS_PATH，这里设为./aux_files/stopwords
STOPWORDS_PATH = './aux_files/stopwords'
stop_words = []                         
with codecs.open(STOPWORDS_PATH, 'r', encoding='utf-8') as fin:        
    for line in fin:
        stop_words.append(line.strip())             

In [14]:
# 对输入的文本进行分词处理，并去除停用词
def clean_text(text, stop_words):          
    """分词，去停用词"""
    cleaned = []
    for w in jieba.lcut(text):
        if len(w.strip()) > 0 and w not in stop_words:
            cleaned.append(w)
    return ' '.join(cleaned)

In [15]:
print('分词和去停用词...')

# 对训练集、验证集和测试集的文本数据进行分词和去除停用词的预处理，并将处理后的数据持久化存储，以便后续重复使用而无需再次执行耗时的预处理步骤。
# 首先检查dat_pkl_path指定的文件是否存在，如果存在，则从该文件中加载之前保存的预处理数据集。
dat_pkl_path = './aux_files/data.pkl'
if os.path.exists(dat_pkl_path):
    print('加载数据...')
    with open(dat_pkl_path, 'rb') as fin:
        train_df, valid_df, test_df = pickle.load(fin)
else:
    # 如果预处理数据不存在，那么对每个数据集的'text'列应用clean_text函数，进行分词和去除停用词的操作。
    # 处理完成后，使用pickle模块将处理后的数据集序列化并保存到dat_pkl_path指定的文件中。
    # 第一次处理过程较长。
    print('新处理数据...')
    train_df['text'] = train_df['text'].apply(lambda x: clean_text(x, stop_words))
    valid_df['text'] = valid_df['text'].apply(lambda x: clean_text(x, stop_words))
    test_df['text'] = test_df['text'].apply(lambda x: clean_text(x, stop_words))    
    # 文件存储
    with open(dat_pkl_path, 'wb') as fout:
        pickle.dump([train_df, valid_df, test_df], fout)
        
train_df.head(3)

Building prefix dict from the default dictionary ...


分词和去停用词...
新处理数据...


Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.673 seconds.
Prefix dict has been built successfully.


,text,target
0,昌宁县 人民检察院 指控 2014 年 月 19 日 下午 16 时许 被告人 段 某驾 拖...,95
1,公诉 机关 指控 2015 年 11 月 10 日晚 时许 被告人 李某 妹妹 李 被害人 ...,95
2,贵州省 平坝县 人民检察院 指控 2014 年 月 日 下午 被告人 王某 丁 堂哥 王 另...,95


In [17]:
train_df.head()

,text,target
0,昌宁县 人民检察院 指控 2014 年 月 19 日 下午 16 时许 被告人 段 某驾 拖...,95
1,公诉 机关 指控 2015 年 11 月 10 日晚 时许 被告人 李某 妹妹 李 被害人 ...,95
2,贵州省 平坝县 人民检察院 指控 2014 年 月 日 下午 被告人 王某 丁 堂哥 王 另...,95
3,审理 查明 2014 年 月 日 14 时许 被告人 叶 驾车 途径 赤壁市 赵李桥镇 胜利...,95
4,安阳县 人民检察院 指控 2014 年 月 27 日 上午 11 时许 宋某 甲 安阳县 吕...,95


In [18]:
valid_df.head()

,text,target
0,公诉 机关 起诉 指控 被告人 张 秘密 窃取 人财物 价值 2210 元 数额较大 已触犯...,108
1,孝昌县 人民检察院 指控 2014 年 月 日 被告人 邬某 孝昌县 城区 路 公交车 被害...,108
2,广东省 广州市 南沙 区 人民检察院 指控 被告人 罗某 2015 年 月 日到 广州市 南...,108
3,公诉 机关 指控 2016 年 月 日 18 时许 被告人 易 行至 达州市 通川区 大观园...,108
4,公诉 机关 指控 1.2015 年 月 20 日 晚上 被告人 胡某 甲 杭州市 淳安县 千...,108


In [19]:
test_df.head()

,text,target
0,公诉 机关 指控 2016 年 月 28 日 20 时许 被告人 颜某 本市 洪山区 马湖 ...,108
1,天津市 静海县 人民检察院 指控 2014 年 月 13 日 上午 时许 被告人 李 xx ...,108
2,永顺县 人民检察院 指控 2014 年 月 11 日 被告人 李 彭 另案处理 永顺县 塔卧...,70
3,公诉 机关 起诉书 指控 2016 年 11 月 17 日 凌晨 时许 被告人 周某 本县 ...,108
4,大名县 人民检察院 起诉书 指控 2014 年 月 25 日 时许 被告人 张某 庄 某处 ...,95


# 训练词向量

In [20]:
# 定义一个函数train_word_embeddings，用于训练Word2Vec词嵌入模型。
# Word2Vec是一种常用的词向量模型，能够将词汇映射到连续的向量空间中，从而捕捉词汇间的语义和语法关系。

def train_word_embeddings(src_data_list, word_embedding_path):
    # 在gensim库的新版本中，size参数已经被替换为vector_size,iter参数也被改为了epochs
    model = Word2Vec(min_count=1, vector_size=100, window=5, sg=1, negative=5, sample=0.001, epochs=10, workers=16)  # 初始化Word2Vec模型 

    # 使用src_data_list中的句子构建词汇表。src_data_list应该是已经分词后的句子列表
    model.build_vocab(corpus_iterable=src_data_list) 

    # 模型训练。继续使用src_data_list训练模型，total_examples和epochs参数用于控制训练过程
    model.train(corpus_iterable=src_data_list, total_examples=model.corpus_count, epochs=model.epochs)

    # 使用save_word2vec_format方法将词向量保存到指定的word_embedding_path文件中，这通常是.txt或.vec格式的文本文件
    model.wv.save_word2vec_format(word_embedding_path)

In [21]:
print('词向量...')
# 首先将训练集、验证集和测试集的文本数据转换为单词列表，然后合并这些列表以准备训练词向量模型。
# 接下来，它检查词向量文件是否存在，如果存在则直接加载，否则训练一个新的词向量模型并保存。
word_embedding_path = './aux_files/own.word2vec'

# 将文本数据转换为单词列表
# 使用split()方法将每条文本数据分割成单词列表，然后将这些列表转换为Python的list类型。
train_list = list(train_df['text'].apply(lambda x: x.split()))   
valid_list = list(valid_df['text'].apply(lambda x: x.split()))
test_list = list(test_df['text'].apply(lambda x: x.split()))

# 合并所有单词列表
# 将三个数据集的单词列表合并为一个大的列表src_data_list，用于词向量模型的训练。
src_data_list = train_list + valid_list + test_list

# 样本个数
print(f'len of src_data_list: {len(src_data_list)}')
# 展示前100个样本
print(src_data_list[100])
    
if os.path.exists(word_embedding_path):   
    print('读取词向量...')
    word_embeddings = KeyedVectors.load_word2vec_format(word_embedding_path, binary=False)
else:                                      # 如果词向量文件不存在，则训练新的词向量模型
    print('训练词向量...')
    train_word_embeddings(src_data_list, word_embedding_path)  
    word_embeddings = KeyedVectors.load_word2vec_format(word_embedding_path, binary=False)
    # 再次加载词向量，这次是为了确保模型已经训练并可以使用
    print('词向量训练完毕...')

词向量...
len of src_data_list: 204231
['审理', '查明', '2013', '年', '月', '日', '下午', '被告人', '田', '某甲', '大城县', '人民检察院', '住宅小区', '院内', '被害人', '乙', '言语', '不合', '厮打', '田', '某甲', '殴打', '乙', '致于', '某乙腰', '处横突', '骨折', '轻伤', '案发后', '田', '某甲', '赔偿', '乙', '经济损失', '乙', '谅解', '2013', '年', '月', '日', '被告人', '田', '某甲', '自动', '公安机关', '投案', '如实', '供认', '被害人', '乙', '事实', '上述事实', '被告人', '田', '某甲', '开庭审理', '过程', '中', '无异议', '被害人', '乙', '陈述', '证人', '某甲', '田某', '乙', '证言', '法医学', '人体', '损伤', '程度', '鉴定书', '协议书', '被告人', '田', '某甲', '投案', '时', '接受', '讯问', '笔录', '证据', '证实', '足以认定', '被告人', '田', '某甲', '居住', '村民', '委员会', '证明', '田', '某甲', '平时', '表现', '无前科', '劣迹', '判处', '本村', '不良影响']
训练词向量...
词向量训练完毕...


In [22]:
word_embeddings['审理']

array([ 0.35918403, -0.04667911,  0.25713038,  0.40582597, -0.24120483,
       -0.02736917,  0.13773754, -0.17984635, -0.08682428,  1.0998052 ,
        0.18352605, -0.56912655, -0.9076452 ,  1.4508265 , -0.30117694,
       -0.30532724, -0.58267844, -0.21307744,  0.8312608 , -0.14654765,
        0.70851016, -0.56026274, -0.3424588 , -0.6085302 , -0.13092417,
        0.44015393, -0.45098317, -0.5192443 , -0.43278164,  0.02986295,
        0.11785586,  0.519084  ,  0.42937672,  0.09964062,  0.40245634,
        0.8120307 ,  0.40212658, -0.04235575,  0.46651635, -0.9146508 ,
       -0.1140155 ,  0.04243552,  0.22194055, -0.51430064, -0.1674125 ,
        1.268741  , -0.24753623, -0.21854803,  0.05382195, -0.45756075,
        0.4305988 ,  0.2658599 ,  0.10162612, -0.07577621, -0.33632684,
       -0.4887352 ,  0.05610463, -0.02146637,  0.324482  , -0.5519011 ,
       -0.27342287, -0.08804538,  0.93529624, -0.39524287,  0.5908051 ,
        0.14378726,  0.09487395,  0.40591815, -0.24234499,  0.12

In [23]:
def build_graph(doc_words_list, weighted_graph = True):   
    """构建图"""
    
    x_adj = []   # 邻接矩阵
    x_feature = []   # 特征向量
    doc_len_list = []
    vocab_set = set()

    # doc_words是一个样本
    for doc_words in doc_words_list:
        
        # 样本长度
        doc_len = len(doc_words)

        # 使用内置函数set()对样本单词去重
        doc_vocab = list(set(doc_words))
        
        # 每个单词是一个node
        doc_nodes = len(doc_vocab)

        doc_len_list.append(doc_nodes)
        vocab_set.update(doc_vocab)

        # word to id
        doc_word_id_map = {}
        for j in range(doc_nodes):
            doc_word_id_map[doc_vocab[j]] = j

        # 滑动窗口
        windows = []
        if doc_len <= window_size:
            windows.append(doc_words)
        else:
            for j in range(doc_len - window_size + 1):
                window = doc_words[j: j + window_size]
                windows.append(window)

        word_pair_count = {}
        for window in windows:
            for p in range(1, len(window)):
                for q in range(0, p):
                    word_p = window[p]
                    word_p_id = word_id_map[word_p]
                    word_q = window[q]
                    word_q_id = word_id_map[word_q]
                    if word_p_id == word_q_id:
                        continue
                    word_pair_key = (word_p_id, word_q_id)
                    # 单词之间的共现作为权值
                    if word_pair_key in word_pair_count:
                        word_pair_count[word_pair_key] += 1.
                    else:
                        word_pair_count[word_pair_key] = 1.
                    # 双向
                    word_pair_key = (word_q_id, word_p_id)
                    if word_pair_key in word_pair_count:
                        word_pair_count[word_pair_key] += 1.
                    else:
                        word_pair_count[word_pair_key] = 1.
    
        row = []
        col = []
        weight = []
        features = []

        for key in word_pair_count:
            p = key[0]
            q = key[1]
            row.append(doc_word_id_map[vocab[p]])
            col.append(doc_word_id_map[vocab[q]])
            weight.append(word_pair_count[key] if weighted_graph else 1.)
        adj = sp.csr_matrix((weight, (row, col)), shape=(doc_nodes, doc_nodes))
    
        # for k, v in sorted(doc_word_id_map.items(), key=lambda x: x[1]):
        #     features.append(word_embeddings.wv[k] if k in word_embeddings.vocab else oov[k])

        for k, v in sorted(doc_word_id_map.items(), key=lambda x: x[1]):   
            if k in word_embeddings.key_to_index:                           # 使用.key_to_index来检查词是否在词汇表中
                features.append(word_embeddings[k])  # 直接使用key访问词向量
            else:
                features.append(oov[k])


        x_adj.append(adj)
        x_feature.append(features)

    
    return x_adj, x_feature

定义参数

In [24]:
class args:
    # 最大遍历次数,表示模型在整个训练数据集上的迭代次数。更多的遍历次数通常意味着模型有更多机会学习数据中的模式，但也可能导致过拟合。
    max_epochs = 30
    # 学习率,是梯度下降算法中的关键参数，决定了权重更新的步长。较高的学习率可以使模型更快收敛，但可能导致训练不稳定；较低的学习率则可能使训练过程缓慢。
    lr = 1e-3
    # 批量大小,批量大小，表示每次更新模型权重时使用的样本数量。较大的批量大小可以加速训练，但可能需要更多的内存；较小的批量大小则可能有助于模型更好地泛化。
    batch_size = 64
    # 词向量维度,词向量维度，表示词嵌入向量的长度。较大的维度可以捕获更多的语义信息，但也会增加模型的复杂度和计算成本。
    embedding_dim = 100
    # GCN隐层神经元数,GCN隐层神经元数，表示图卷积网络中隐层的宽度。这直接影响模型的表达能力和计算复杂度。
    hidden_dim = 128
    # GAT的head参数,GAT的头参数，表示在图注意力网络中并行使用的注意力头的数量。多个头允许模型关注输入的不同部分，从而增强模型的表达能力。
    num_heads = 8
    # 滑动窗口大小,滑动窗口大小，用于确定构建图结构时考虑的上下文范围。较大的窗口可以捕获更长距离的依赖关系，但也可能引入噪音。
    window_size = 3
    # 模型保存路径，指定了训练好的模型将被保存的位置。这对于后续的模型复用和部署非常重要。
    model_save_path = './model'

In [25]:
import time    # 为了计算build_graph函数的执行时间

# 1、构建词汇表

word_embeddings_dim = args.embedding_dim

# 初始化一个空集合word_set，用于存储所有文档中出现的唯一单词。
word_set = set() 

# 遍历所有样本
# 遍历src_data_list（假设这是包含所有文档的列表），更新word_set以包含所有文档中的单词
for doc_words in src_data_list:
    word_set.update(doc_words)

# 将word_set转换为列表vocab，并计算其大小vocab_size
vocab = list(word_set)
vocab_size = len(vocab)

# 2、单词到ID的映射，word to id
word_id_map = {}   
# 通过遍历vocab列表，为每个单词分配一个从0开始的连续整数ID
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

# 3、初始化OOV（Out-Of-Vocabulary）词向量
# 设置低频词典(oov)，创建一个字典oov，用于存储不在预训练词向量模型中的单词的随机初始化向量
oov = {}
# 对于vocab中的每个单词，生成一个维度为word_embeddings_dim的随机向量，范围在-0.1到0.1之间
for v in vocab:
    oov[v] = np.random.uniform(-0.1, 0.1, word_embeddings_dim)


# 构建图结构    
window_size = args.window_size

# build_graph函数调用前后添加了计时代码。start_time记录函数调用前的时间戳，end_time记录函数调用完成后的时刻，两者之差即为函数的执行时间。
start_time = time.time()
print('构建 training 图...')
train_x_adj, train_x_feature = build_graph(train_list, weighted_graph = True)
end_time = time.time()
assert len(train_x_adj) == len(train_x_feature) == len(train_list)
print(f"训练图构建完成，耗时：{end_time - start_time:.2f} 秒")

start_time = time.time()
print('构建 valid 图...')
valid_x_adj, valid_x_feature = build_graph(valid_list, weighted_graph = True)
end_time = time.time()
assert len(valid_x_adj) == len(valid_x_feature) == len(valid_list)
print(f"训练图构建完成，耗时：{end_time - start_time:.2f} 秒")

start_time = time.time()
print('构建 test 图...')
test_x_adj, test_x_feature = build_graph(test_list, weighted_graph = True)
end_time = time.time()
assert len(test_x_adj) == len(test_x_feature) == len(test_list)
print(f"训练图构建完成，耗时：{end_time - start_time:.2f} 秒")

构建 training 图...
训练图构建完成，耗时：165.38 秒
构建 valid 图...
训练图构建完成，耗时：16.55 秒
构建 test 图...
训练图构建完成，耗时：31.86 秒


# 准备训练
准备图数据集

In [26]:
# GraphDataset类继承自DGLDataset，这是DGL（Deep Graph Library）框架中的一个基类，用于处理图数据集。
# 这个类主要用于封装图数据及其对应的标签，以便于在PyTorch或其它深度学习框架中使用。
class GraphDataset(DGLDataset):
    """图数据集"""    

    # 1、初始化函数
    def __init__(self, x_adj, x_feature, targets = None):
        self.adj_matrix = x_adj
        self.node_matrix = x_feature
        self.targets = targets

    # 2、长度函数：
    def __len__(self):
        return len(self.adj_matrix)

    # 3、获取项函数：
    def __getitem__(self, idx):
        scipy_adj = self.adj_matrix[idx]
        G = dgl.from_scipy(scipy_adj)
        G.ndata['feat'] = torch.stack([torch.tensor(x, dtype = torch.float) for x in self.node_matrix[idx]])
        if self.targets is not None:
            label = self.targets[idx]
            return G, torch.tensor(label, dtype = torch.long)
        return G

定义模型

In [27]:
# GCNClassifier类是一个基于图卷积网络（GCN）的分类器，继承自PyTorch的nn.Module。这个类实现了两层GCN卷积，随后是平均池化、Dropout和全连接层，用于最终的分类任务。
class GCNClassifier(nn.Module):
    """GCN"""    
    # 初始化函数：
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(GCNClassifier, self).__init__()
        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.avgpooling = AvgPooling()
        self.drop = nn.Dropout(p = 0.3)
        self.classify = nn.Linear(hidden_dim, n_classes)

    # 前向传播函数：
    def forward(self, g, h):
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        h = self.drop(h)
        h = self.avgpooling(g, h)
        return self.classify(h)

In [28]:
# GATClassifier类是一个基于图注意力网络（GAT）的分类器，同样继承自PyTorch的nn.Module。
# 这个类通过多头注意力机制来增强图卷积的效果，适用于处理图结构数据的分类任务。
class GATClassifier(nn.Module):
    """GAT"""    
    # 初始化函数：
    def __init__(self, in_dim, hidden_dim, num_heads, n_classes):
        super(GATClassifier, self).__init__()
        self.hid_dim = hidden_dim
        self.gat1 = GATConv(in_dim, hidden_dim, num_heads)
        self.gat2 = GATConv(hidden_dim*num_heads, hidden_dim, 1)
        self.avgpooling = AvgPooling()
        self.drop = nn.Dropout(p = 0.3)
        self.classify = nn.Linear(hidden_dim, n_classes)

    # 前向传播函数：
    def forward(self, g, h):
        # batch size批量大小
        bs = h.shape[0]
        h = F.relu(self.gat1(g, h))
        h = h.reshape(bs, -1)
        h = F.relu(self.gat2(g, h))
        h = h.reshape(bs, -1)
        h = self.drop(h)
        h = self.avgpooling(g, h)
        return self.classify(h)

# 训练

In [29]:
# 这段代码定义了一个训练流程，用于训练图神经网络模型，如GCN或GAT
def train(args, model, train_info, val_info, num_classes, model_type):
    """训练函数"""

    # 1、数据准备
    # 从train_info和val_info中提取训练集和验证集的邻接矩阵、节点特征和标签列表。
    train_adj_list, train_node_list, train_label_list = train_info
    val_adj_list, val_node_list, val_label_list = val_info

    # 创建GraphDataset实例，分别封装训练集和验证集的数据
    traindataset = GraphDataset(train_adj_list, train_node_list, train_label_list)
    valdataset = GraphDataset(val_adj_list, val_node_list, val_label_list)
    
    # shuffle = True：在一个epoch之后，对所有的数据随机打乱，再按照设定好的每个批次的大小划分批次
    trainloader = GraphDataLoader(traindataset, batch_size = args.batch_size, shuffle = True)  
    valloader = GraphDataLoader(valdataset, batch_size = args.batch_size, shuffle = False)

    # 2、定义损失函数和优化器
    criterion = CrossEntropyLoss()   # 交叉熵损失函数函数
    optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)    # 优化器

    # 训练循环
    best_val_f1 = 0

    # 遍历args.max_epochs指定的epoch数
    for idx in range(args.max_epochs):
        print(f'Epoch {idx + 1}/{args.max_epochs}')

        # 每个epoch中，先调用train_one_epoch函数进行模型训练，返回训练损失、准确率、F1分数和AUC值
        train_loss, train_acc, train_f1, train_auc = train_one_epoch(trainloader, model, criterion, optimizer, num_classes)
        
        # 然后调用validate函数在验证集上评估模型性能，返回验证损失、准确率、F1分数和AUC值。
        val_loss, val_acc, val_f1, val_auc = validate(valloader, model, criterion, num_classes)
        
        # 打印每个epoch的训练和验证指标
        print('train_loss: {}, train_f1: {}, train_auc: {}, train_acc: {}, val_loss: {}, val_f1: {}, val_auc: {}, val_acc: {}'.
              format(train_loss, train_f1, train_auc, train_acc, val_loss, val_f1, val_auc, val_acc))
        
        # 保存f1值最好的epoch的模型用于测试
        # 如果当前验证集的F1分数高于历史最佳，保存当前模型状态到磁盘，以备后续测试使用
        if val_f1 > best_val_f1:
            print(f'----- Save model ----- with f1: {val_f1}')
            torch.save(model.state_dict(), f'{args.model_save_path}/{model_type}-epoch-{idx}-{val_f1}.pt')
            best_val_f1 = val_f1

In [30]:
# 这段代码定义了train_one_epoch函数，用于执行模型在一个epoch上的训练过程。
def train_one_epoch(trainloader, model, criterion, optimizer, num_classes):
    """训练一个 epoch"""

# 1、初始化变量：
    # 初始化train_loss、train_f1和train_auc变量，用于累计整个epoch的训练损失、F1分数和AUC值。
    train_loss = 0
    train_f1 = 0
    train_auc = 0
    # 创建两个空列表all_labels和all_logits，用于存储所有批次的真实标签和模型预测的logits。
    all_labels = []
    all_logits = []

# 2、模型训练
    total = len(trainloader)
    model.train()
    for idx, (G, label) in tqdm(enumerate(trainloader), total = total):
                
        h = G.ndata['feat'].float()
        logit = model(G, h)
        loss = criterion(logit, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   # 反向传播梯度更新
        
        label_numpy = label.detach().cpu().numpy()
        logit_numpy = logit.softmax(-1).detach().cpu().numpy()
        
        train_loss += loss.item()/total
        # train_f1 += f1_score(label_numpy, logit_numpy.argmax(-1), average = 'micro')/total
        
        all_labels.append(label_numpy)
        all_logits.append(logit_numpy)

# 3、性能评估
    # 在每个批次结束后，收集所有批次的真实标签和预测logits。
    all_labels = np.concatenate(all_labels)
    all_logits = np.concatenate(all_logits)

    # 计算整个epoch的AUC值，使用roc_auc_score函数，参数multi_class='ovo'表示采用一对多策略计算多分类问题的AUC。
    train_auc = roc_auc_score(all_labels, all_logits, multi_class = 'ovo', labels = np.array([int(i) for i in range(num_classes)]))

     # 计算准确率和加权平均F1分数，使用accuracy_score和f1_score函数，其中f1_score的average='weighted'参数表示计算加权平均F1分数，权重由每个类别的样本数量决定。
    all_logits_label = np.argmax(all_logits, 1)
    train_acc = accuracy_score(all_labels, all_logits_label)
    train_f1 = f1_score(all_labels, all_logits_label, average = 'weighted')

# 4、返回结果
    # 返回整个epoch的平均训练损失、准确率、F1分数和AUC值。 
    return train_loss, train_acc, train_f1, train_auc

In [31]:
# 这段代码定义了validate函数，用于在验证集上评估模型的性能
def validate(valloader, model, criterion, num_classes):
    """在验证集上验证"""

# 1、初始化变量：
    # 初始化val_loss、val_f1和val_auc变量，用于累计整个验证集的损失、F1分数和AUC值。
    # 创建两个空列表all_labels和all_logits，用于存储所有批次的真实标签和模型预测的logits。
    val_loss = 0
    val_f1 = 0
    val_auc = 0    
    all_labels = []
    all_logits = []

# 2、模型评估：
    total = len(valloader)
    model.eval()
    
    with torch.no_grad():
        for idx, (G, label) in tqdm(enumerate(valloader), total = total):

            h = G.ndata['feat'].float()
            logit = model(G, h)
            loss = criterion(logit, label)

            label_numpy = label.detach().cpu().numpy()
            logit_numpy = logit.softmax(-1).detach().cpu().numpy()

            val_loss += loss.item()/total
            # val_f1 += f1_score(label_numpy, logit_numpy.argmax(-1), average = 'micro')/total

        
            all_labels.append(label_numpy)
            all_logits.append(logit_numpy)

        all_labels = np.concatenate(all_labels)
        all_logits = np.concatenate(all_logits)

# 3、性能评估：
    # 计算整个验证集的AUC值，使用roc_auc_score函数，参数multi_class='ovo'表示采用一对多策略计算多分类问题的AUC。           
        val_auc = roc_auc_score(all_labels, all_logits, multi_class = 'ovo', labels = np.array([int(i) for i in range(num_classes)]))
        all_logits_label = np.argmax(all_logits, 1)
        
    # 计算准确率和微平均F1分数，使用accuracy_score和f1_score函数，其中f1_score的average='micro'参数表示计算微平均F1分数，这适用于不平衡数据集的情况。 
        val_acc = accuracy_score(all_labels, all_logits_label)
        val_f1 = f1_score(all_labels, all_logits_label, average = 'micro')
    
    # 返回整个验证集的平均损失、准确率、F1分数和AUC值。
    return val_loss, val_acc, val_f1, val_auc

## GAT 模型

In [32]:
# 这段代码初始化了GAT分类器模型，并调用了训练函数train来在训练集和验证集上进行模型训练。

# 1、确定类别数量：
num_classes = len(label2idx)

# 2、模型初始化：
model = GATClassifier(args.embedding_dim, args.hidden_dim, args.num_heads, num_classes)

# 3、打印训练信息：
print('-------- GAT 模型训练')

# 4、调用训练函数
train(args, model,                                                 
      (train_x_adj, train_x_feature, train_df['target'].values),   
      (valid_x_adj, valid_x_feature, valid_df['target'].values),   
      num_classes, 
      "GAT")                                          

-------- GAT 模型训练
Epoch 1/30


100%|██████████| 268/268 [00:15<00:00, 17.07it/s]


train_loss: 1.0922293572853141, train_f1: 0.7022220846756022, train_auc: 0.9467752668313448, train_acc: 0.714901159180294, val_loss: 0.6782911906936274, val_f1: 0.8077753779697624, val_auc: 0.9916523396579271, val_acc: 0.8077753779697624
----- Save model ----- with f1: 0.8077753779697624
Epoch 2/30


100%|██████████| 268/268 [00:13<00:00, 19.58it/s]


train_loss: 0.7025283856243393, train_f1: 0.7991066406262641, train_auc: 0.9856910394435883, train_acc: 0.8058178948457876, val_loss: 0.6155361468806418, val_f1: 0.8277975599789855, val_auc: 0.9942351163035886, val_acc: 0.8277975599789855
----- Save model ----- with f1: 0.8277975599789855
Epoch 3/30


...

train_loss: 0.15658065229530907, train_f1: 0.9520211711359359, train_auc: 0.9998559094907327, train_acc: 0.9520803146346513, val_loss: 0.6473996080041154, val_f1: 0.8540657287957504, val_auc: 0.9958372175987152, val_acc: 0.8540657287957504
Epoch 29/30


100%|██████████| 268/268 [00:13<00:00, 19.59it/s]


train_loss: 0.14764000437282548, train_f1: 0.9545307895881741, train_auc: 0.9998630164283239, train_acc: 0.9545901469675016, val_loss: 0.7203726787050605, val_f1: 0.8466522678185745, val_auc: 0.9952422146848514, val_acc: 0.8466522678185745
Epoch 30/30


100%|██████████| 268/268 [00:13<00:00, 19.59it/s]


train_loss: 0.13979492432727714, train_f1: 0.9563583696188133, train_auc: 0.9998928247647824, train_acc: 0.9564143034568412, val_loss: 0.7137164866774163, val_f1: 0.8479364894051719, val_auc: 0.9954369422614489, val_acc: 0.8479364894051719


## GCN 模型

In [33]:
# 这段代码初始化了GCN分类器模型，并调用了训练函数train来进行模型训练。

# 1、确定类别数量：
num_classes = len(label2idx)

# 2、模型初始化：
model = GCNClassifier(args.embedding_dim, args.hidden_dim, num_classes)

# 3、打印训练信息：
print('-------- GCN 模型训练')

# 4、调用训练函数：
train(args, model,                                                 
      (train_x_adj, train_x_feature, train_df['target'].values),   
      (valid_x_adj, valid_x_feature, valid_df['target'].values),   
      num_classes,                                                 
      "GCN")

-------- GCN 模型训练
Epoch 1/30


100%|██████████| 268/268 [00:12<00:00, 21.35it/s]


train_loss: 1.70563102121286, train_f1: 0.5420636602951951, train_auc: 0.9083831769678864, train_acc: 0.56693748706272, val_loss: 1.0791760883224542, val_f1: 0.692720798552332, val_auc: 0.9769426481044641, val_acc: 0.692720798552332
----- Save model ----- with f1: 0.692720798552332
Epoch 2/30


100%|██████████| 268/268 [00:12<00:00, 21.35it/s]


train_loss: 1.0208729879424858, train_f1: 0.6973118519205526, train_auc: 0.9750637048755078, train_acc: 0.7120226143655558, val_loss: 0.8834435901701899, val_f1: 0.7512112544509952, val_auc: 0.9872050748319297, val_acc: 0.7512112544509952
----- Save model ----- with f1: 0.7512112544509952
Epoch 3/30


...

train_loss: 0.5315739883554002, train_f1: 0.8461081819972337, train_auc: 0.9970663650796853, train_acc: 0.8482715793831505, val_loss: 0.5658971046639693, val_f1: 0.8349191524137528, val_auc: 0.9958888201925733, val_acc: 0.8349191524137528
Epoch 29/30


100%|██████████| 268/268 [00:12<00:00, 21.70it/s]


train_loss: 0.5259153211662021, train_f1: 0.847874047809756, train_auc: 0.9971989633679982, train_acc: 0.8500245808321258, val_loss: 0.5664621611584479, val_f1: 0.8371957270445392, val_auc: 0.9959073274981556, val_acc: 0.8371957270445392
Epoch 30/30


100%|██████████| 268/268 [00:12<00:00, 21.73it/s]


train_loss: 0.5214620593269136, train_f1: 0.8494699363666118, train_auc: 0.9971907440039245, train_acc: 0.851551179879942, val_loss: 0.5755264366796217, val_f1: 0.8336349308271555, val_auc: 0.9959132521428402, val_acc: 0.8336349308271555


# 预测

In [40]:
# 这段代码定义了一个load_model函数，用于加载预训练的模型权重，并将模型设置为评估模式
def load_model(model, best_model_path):
    """加载模型、加载模型权重"""
    model.load_state_dict(torch.load(best_model_path))

    # 调用model.eval()将模型设置为评估模式。在评估模式下，模型的一些行为会有所不同，
    model.eval()
    return model 

In [41]:
# 这段代码定义了test函数，用于在测试集上评估模型的性能。
def test(args, model, x_adj, x_feature, label_list, num_classes):
    """测试"""
# 1、创建测试数据集和数据加载器
    testdataset = GraphDataset(x_adj, x_feature)
    testloader = GraphDataLoader(testdataset, batch_size = args.batch_size, shuffle = False)

    # 2、模型预测
    pred_list = []
    with torch.no_grad():                             
        for idx, G in enumerate(tqdm(testloader)):    
            h = G.ndata['feat'].float()              
            log = model(G, h)                         
            logits = log.softmax(-1)                  
            pred_soft = logits.detach().cpu().numpy() 
            pred_list.append(np.argmax(pred_soft, 1)) 

        preds = np.concatenate(pred_list)             

    # 计算加权平均F1分数、精确度、召回率和准确率，使用f1_score、precision_score、recall_score和accuracy_score函数，
    # 其中average='weighted'参数表示计算加权平均值，权重由每个类别的样本数量决定
    print('F1 in test:', f1_score(label_list, preds, average = 'weighted'))
    print('precision in test:', precision_score(label_list, preds, average = 'weighted'))
    print('recall in test:', recall_score(label_list, preds, average = 'weighted'))
    print('accuracy in test:', accuracy_score(label_list, preds))

## GAT

In [42]:
# 1、确定类别数量：num_classes变量通过计算label2idx字典的长度来确定类别数量。 
num_classes = len(label2idx)

# 2、模型初始化和加载
model = GATClassifier(args.embedding_dim, args.hidden_dim, args.num_heads, num_classes)

# 定义best_model_path变量，指向保存最佳模型权重的文件路径。
best_model_path = f'{args.model_save_path}/GAT-epoch-13-0.8641060066546028.pt'

# 调用load_model函数，加载best_model_path中的模型权重到model实例上，并将模型设置为评估模式
model = load_model(model, best_model_path)

# 调用test函数，传入参数args（包含训练配置）、model（加载了最佳权重的GAT分类器模型）、测试集的邻接矩阵test_x_adj、
test(args, model, test_x_adj, test_x_feature, test_df['target'].values, len(label2idx))

100%|██████████| 508/508 [00:26<00:00, 18.85it/s]
/home/sunmr/anaconda3/envs/sun/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


F1 in test: 0.828065190694157
precision in test: 0.8405425486020889
recall in test: 0.834963701242771
accuracy in test: 0.834963701242771


## GCN

In [43]:
# 1、模型初始化：创建GCNClassifier实例，传入参数args.embedding_dim（输入特征维度）、args.hidden_dim（隐藏层维度）和num_classes（类别数量）。
model = GCNClassifier(args.embedding_dim, args.hidden_dim, num_classes)

# 2、加载最佳模型权重
best_model_path = f'{args.model_save_path}/GCN-epoch-22-0.8372541007530209.pt'
model = load_model(model, best_model_path)

# 3、用test函数，传入参数args（包含训练配置）、model（加载了最佳权重的GCN分类器模型）、测试集的邻接矩阵test_x_adj、
test(args, model, test_x_adj, test_x_feature, test_df['target'].values, len(label2idx))

100%|██████████| 508/508 [00:24<00:00, 20.56it/s]

F1 in test: 0.7988395395765523
precision in test: 0.8130721153032072
recall in test: 0.8079242032730405
accuracy in test: 0.8079242032730405



/home/sunmr/anaconda3/envs/sun/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
